**ADDIITIONAL FILES 1 - 22**

In [1]:
# import libraries
import os.path
import pandas as pd
import os
from pandas.api.types import is_object_dtype, is_numeric_dtype, is_bool_dtype, is_string_dtype, is_float_dtype
import numpy as np
import re
import glob
import openpyxl
import os.path
from openpyxl.styles import PatternFill
import traceback
from openpyxl import Workbook
import datetime
import argparse

C:\Users\Nahila.Chowdhury\AppData\Local\Temp\ipykernel_26400\4125756671.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
green_fill = PatternFill(start_color='72C931', end_color='72C931', fill_type='solid')
red_fill = PatternFill(start_color='FABF8F', end_color='FABF8F', fill_type='solid')

#### File location and names
---

In [21]:
error_output_path = r"C:\Users\Nahila.Chowdhury\OneDrive - OFWAT\Data\Business Plan Tables\August 2024 BPT Submission\error\proteus error\Additional tables\\"
input_files = r"C:\Users\Nahila.Chowdhury\OneDrive - OFWAT\Data\Business Plan Tables\August 2024 BPT Submission\3_Files error fixing WORKING\Additional Tables"
v7_bpt = r"C:\Users\Nahila.Chowdhury\OneDrive - OFWAT\Data\Business Plan Tables\August 2024 BPT Submission\1_Files as submitted\Additional Tables\Copy of PR24 NEW BPDT's for Post DD collection - Master.xlsx"

#### Code
---

In [4]:
def main(dict_of_sheets, original_dict_of_sheets, excel_error_log_name):
    firstsheet = excel_error_log_name['Intro']
    try:
        for key, value in dict_of_sheets.items():
            if value[1] == "fOut_":
                if fOut_headers_consistency(key, value[0], excel_error_log_name):  # Rule  1
                    regex_boncode(key,value[0],excel_error_log_name) #Rule 2
                    #check_suffix(key,value[0],excel_error_log_name) #Rule 3
                    check_data_type(key,value[0],excel_error_log_name) #Rule 4
                    check_ref(key,value[0],excel_error_log_name) #Rule 5
                    user_text_input(key,value[0],excel_error_log_name) #Rule 6
                    boncode_Consistency(key,value[0],original_dict_of_sheets[key][0],excel_error_log_name) #Rule 7
                    boncode_Uniqueness(key, value[0],excel_error_log_name) #Rule 8
                    user_numerical_input(key,value[0],excel_error_log_name) #Rule 9
                    percentage_range(key,value[0],excel_error_log_name) #Rule 10
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {ValueError}"
        pass 

In [5]:
##Import Company excel file
def import_data(file_location, excel_error_log_name):
    firstsheet = excel_error_log_name['Intro']
    try:
        xl1 = pd.ExcelFile(file_location)  # Define the excel file
        worksheets = xl1.sheet_names  # Get the list of worksheets in the file
        found_dictionary, found_f_outputs = (False, False)
        dict_of_sheets = {}
        for sheet in worksheets:  # iterate through the sheets in the file
            if sheet.startswith("Dict_"):  # ...if the worksheet starts with the text "Dict_" then...
                found_dictionary = True
                dict_of_sheets[sheet] = (pd.read_excel(xl1, sheet_name=sheet, skiprows=[0, 2]), "Dict_")
            if sheet.startswith("fOut_"):  # ...if the workseet starts with the text "fOut" then...
                found_f_outputs = True
                dict_of_sheets[sheet] = (pd.read_excel(xl1, sheet_name=sheet, skiprows=[0, 2]), "fOut_")
        if not found_f_outputs:
            firstsheet['B6'].value = f"{firstsheet['B6'].value} Error message: A worksheet starting with 'fOut_' (an F_Outputs sheet) was not found in worksheets, Proteus has failed to run"
            firstsheet['B6'].fill = red_fill
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {ValueError}"
        firstsheet['B6'].fill = red_fill
        pass
    return dict_of_sheets

In [6]:
##Import original excel file : V7
def import_original_data(original_file_location, excel_error_log_name):
    firstsheet = excel_error_log_name['Intro']
    try:
        xl1 = pd.ExcelFile(original_file_location)  # Define the excel file
        worksheets = xl1.sheet_names  # Get the list of worksheets in the file
        original_dict_of_sheets = {}
        for sheet in worksheets:  # iterate through the sheets in the file
            if sheet.startswith("Dict_"):  # ...if the workseet starts with the text "Dict_" then...
                original_dict_of_sheets[sheet] = (pd.read_excel(xl1, sheet_name=sheet, skiprows=[0, 2]), "Dict_")
            if sheet.startswith("fOut_"):  # ...if the workseet starts with the text "fOut" then...
                excel_error_log_name.create_sheet(title=sheet[5:] + ' Output')  # creating the sheets which will house the tables of outputs
                original_dict_of_sheets[sheet] = (pd.read_excel(xl1, sheet_name=sheet, skiprows=[0, 2]), "fOut_")
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {ValueError}"
        firstsheet['B6'].fill = red_fill
        pass
    return original_dict_of_sheets

In [7]:
def timestamp_error_log(excel_error_log_name):
    firstsheet = excel_error_log_name['Intro']
    try:
        # storing current date and time
        current_date_time = datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")
        firstsheet['B4'].value = f"This file was created at: {current_date_time}"
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {ValueError}"
        firstsheet['B6'].fill = red_fill
        pass

In [8]:
def fOut_headers_consistency(sheet_name, df, excel_error_log_name):
    ## ---------------------------------------------------------------------
    ## *Headers* in spesific order and value. Headers are case insensitive
    ## ----------------------------------------------------------------------
    firstsheet = excel_error_log_name['Intro']
    try:
        outputsheet = excel_error_log_name[sheet_name[5:] + ' Output'] #defining which sheet the output needs to go into
        outputsheet['B3'].value = 'Data Validation Rule 1'
        outputsheet['C3'].value = ""

        dictionary_errors_column_headers = {
            'Acronym': (df.columns[0],
                        "Error message: 'Acronym' column name is not correct, please note name is case sensitive"),
            'Reference': (df.columns[1],
                          "Error message: 'Reference' column name is not correct, please note name is case sensitive"),
            'Item description': (df.columns[2],
                                 "Error message: 'Item description' column name is not correct, please note name is case sensitive"),
            'Unit': (
            df.columns[3], "Error message: 'Unit' column name is not correct, please note name is case sensitive"),
            'Model': (
            df.columns[4], "Error message: 'Model' column name is not correct, please note name is case sensitive"),
        }

        error_counter = 0
        for key, value in dictionary_errors_column_headers.items():
            if key == value[0]:
                pass
            else:
                error_counter += 1
                outputsheet['C3'].value = f"{outputsheet['C3'].value} {value[1]}"
                outputsheet['C3'].fill = red_fill
        if error_counter == 0:
            outputsheet['C3'].value = "Success, No Errors detected!"
            outputsheet['C3'].fill = green_fill
            return True
        outputsheet['C3'].value = f"{outputsheet['C3'].value} Please correct the header's names for this Excel Sheet: {sheet_name}, before data validation proceeds!"
        outputsheet['C3'].fill = red_fill
        return False
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {ValueError}"
        firstsheet['B6'].fill = red_fill
        return False


In [9]:
def regex_boncode(sheet_name,df,excel_error_log_name):
##-------------------------------------------------------------------------------------------------------
##Boncode find pattern using *Regex*. Reference (BON code) should consist of the following in sequence:
##An upper case letter. Zero or more upper case letters or underscores.
## At least one digit. Zero or more number upper case letters, underscores, or digits.
##No lower case letters. Boncodes must not contain "-" character.
##Note "str.fullmatch" doesn't work due to older python vesrion installed
##-------------------------------------------------------------------------------------------------------
    firstsheet = excel_error_log_name['Intro']
    try:
        outputsheet = excel_error_log_name[sheet_name[5:] + ' Output']  # defining which sheet the output needs to go into
        outputsheet['B4'].value = 'Data Validation Rule 2'
        outputsheet['C4'].value = ""
        index=3 #index for showing cell position
        regex = r'^([A-Z][A-Z_]*[0-9]+[A-Z0-9_]*)$'
        regex_pattern = re.compile(regex)
        error_counter=0
        for item in df['Reference']:
            index+=1
            if pd.isna(item):
                error_counter+=1
                outputsheet['C4'].value = f"{outputsheet['C4'].value} Error in Row {index}: 'Reference': {item}, 'Reference' is a mandatory field and cannot have empty values \n"
                outputsheet['C4'].fill = red_fill
                break
            if bool(regex_pattern.match(item))==False:
                error_counter+=1
                outputsheet['C4'].value = f"{outputsheet['C4'].value} Error in Row {index}: 'Reference': {item} doesn't match the regular expression \n"
                outputsheet['C4'].fill = red_fill
        if error_counter==0:
            outputsheet['C4'].value = "Success, No Errors detected!"
            outputsheet['C4'].fill = green_fill
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {ValueError} \n"
        firstsheet['B6'].fill = red_fill
        pass

In [10]:
def check_suffix(sheet_name, df, excel_error_log_name):
    ## ---------------------------------------
    ## Checking *suffix _PR24* on Boncodes
    ## ---------------------------------------
    firstsheet = excel_error_log_name['Intro']
    try:
        outputsheet = excel_error_log_name[sheet_name[5:] + ' Output']  # defining which sheet the output needs to go into
        outputsheet['B5'].value = 'Data Validation Rule 3'
        outputsheet['C5'].value = ""
        index = 3  # index for showing row position
        suffix = "_PR24"
        error_counter = 0

        for item in df['Reference']:
            index += 1
            if pd.notna(item):
                if item.endswith(suffix):
                    pass
                else:
                    error_counter += 1
                    outputsheet['C5'].value = f"{outputsheet['C5'].value} Error in Row {index}: Reference: {item} does not has a suffix _PR24\n"
                    outputsheet['C5'].fill = red_fill
        if error_counter == 0:
            outputsheet['C5'].value ="Success, No Errors detected!"
            outputsheet['C5'].fill = green_fill
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {ValueError} \n"
        firstsheet['B6'].fill = red_fill
        pass

In [11]:
def check_data_type(sheet_name, df, excel_error_log_name):
    ## ------------------------------------------------------------------------------------------
    ##6. *Data Type Checking*. This verifies that the entered data has the appropriate data type.
    ## ------------------------------------------------------------------------------------------
    # Unit: Unit must be less than 21 characters.
    # Description: Description must be less than 230 characters.
    regex_columns = r'^([0-9]{4}-[0-9]{2})$'
    regex_pattern = re.compile(regex_columns)
    firstsheet = excel_error_log_name['Intro']
    try:
        outputsheet = excel_error_log_name[sheet_name[5:] + ' Output']  # defining which sheet the output needs to go into
        outputsheet['B6'].value = 'Data Validation Rule 4'
        outputsheet['C6'].value = ""
        index = 3  # index for showing row position
        error_counter = 0
        unit_length = 21
        description_length = 230

        for item1, item2, item3 in zip(df['Unit'], df['Reference'],df['Item description']):
            index += 1
            if type(item1) == int:
                error_counter += 1
                outputsheet['C6'].value = f"{outputsheet['C6'].value} Error in Row {index}: Reference: {item2}: Unit must not be a number \n"
                outputsheet['C6'].fill = red_fill
            elif pd.notna(item1):
                if len(item1) > unit_length:
                    error_counter += 1
                    outputsheet['C6'].value = f"{outputsheet['C6'].value} Error in Row {index}: Reference: {item2}: Unit must be <={unit_length} characters \n"
                    outputsheet['C6'].fill = red_fill
            elif pd.notna(item3):
                if len(item3) > description_length:
                    error_counter += 1
                    outputsheet['C6'].value = f"{outputsheet['C6'].value} Error in Row {index}: Reference: {item2}: Description must be <={description_length} characters \n"
                    outputsheet['C6'].fill = red_fill
        index = 3
        for i, row in df.iterrows():
            index += 1
            for key in row.keys():
                if bool(regex_pattern.match(key)) or key.lower() == 'constant':
                    if len(str(row[key])) > description_length:
                        error_counter +=1
                        outputsheet['C6'].value = f"{outputsheet['C6'].value}Error in Row: {index}, Reference: {df.loc[i, 'Reference']},Column: {key}, Value: {row[key]}- this value is over 230 characters.\n\n"
                        outputsheet['C6'].fill = red_fill
        if error_counter == 0:
            outputsheet['C6'].value = "Success, No Errors detected!"
            outputsheet['C6'].fill = green_fill
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {ValueError} \n"
        firstsheet['B6'].fill = red_fill
        pass

In [12]:
def check_ref(sheet_name, df, excel_error_log_name):
    # REF! error
    firstsheet = excel_error_log_name['Intro']
    regex_columns = r'^([0-9]{4}-[0-9]{2})$'
    regex_pattern = re.compile(regex_columns)
    
    try:
        outputsheet = excel_error_log_name[sheet_name[5:] + ' Output']  # defining which sheet the output needs to go into
        outputsheet['B7'].value = 'Data Validation Rule 5'
        outputsheet['C7'].value = ""
        index = 3
        error_counter = 0

        for i, row in df.iterrows():
            index += 1
            if row['Model'] and pd.isna(row['Reference']) and pd.isna(row['Unit']):
                error_counter += 1
                outputsheet['C7'].value = f"{outputsheet['C7'].value} Error in Row: {index}," \
                    "This likely contains a reference error, please check \n"
                outputsheet['C7'].fill = red_fill
        if error_counter == 0:
            outputsheet['C7'].value = "Success, No Errors detected!"
            outputsheet['C7'].fill = green_fill
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {str(ValueError)} \n"
        firstsheet['B6'].fill = red_fill
        pass

In [13]:
def boncode_Uniqueness(sheet_name,df,excel_error_log_name):
#-----------------------------------------------------------------------------------
#Boncodes Uniqueness*: same boncode has been used only once in the dictionary file 
#Identify any duplicated Boncodes in across the dictionary
#-----------------------------------------------------------------------------------
#add items one by one to a list and while adding check if it is duplicated or not
    firstsheet = excel_error_log_name['Intro']
    try: 
        outputsheet = excel_error_log_name[sheet_name[5:] + ' Output']  # defining which sheet the output needs to go into
        outputsheet['B10'].value = 'Data Validation Rule 8'
        index=3 #index for showing row position
        boncode_duplicates = []
        error_counter=0
        
        for item in df['Reference']:  
            index += 1
            if item in boncode_duplicates:
                error_counter += 1
                outputsheet['C10'].value = f"Error in row {index}: Reference: {item} is not unique"
                outputsheet['C10'].fill = red_fill
            else: 
                boncode_duplicates.append(item)
        if error_counter == 0:
            outputsheet['C10'].value = "Success, No Errors detected!"
            outputsheet['C10'].fill = green_fill
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value}{ValueError} \n"
        firstsheet['B6'].fill = red_fill
        pass

In [14]:
def user_text_input(sheet_name, df, excel_error_log_name):
    # ---------------------------------------------------------------------
    ##Whether the values (e.g. anything under the years headers) corresponds to what is in the units columns
    # If the Unit value is not ‘Text’ the years columns values should be number
    #sheet RR 414-419, 489-491 error because unit is weird
    # ---------------------------------------------------------------------
    regex_columns = r'^([0-9]{4}-[0-9]{2})$'
    regex_pattern = re.compile(regex_columns)

    firstsheet = excel_error_log_name['Intro']
    try:
        outputsheet = excel_error_log_name[sheet_name[5:] + ' Output']  # defining which sheet the output needs to go into
        outputsheet['B8'].value = 'Data Validation Rule 6 (will not stop data loading to fountain)'
        outputsheet['C8'].value = ""
        index = 3
        error_counter = 0

        for i, row in df.iterrows():
            index += 1
            for key in row.keys():
                if bool(regex_pattern.match(key)) or key.lower() == 'constant':
                    if row[key] == "##BLANK":
                        continue
                    elif row[key] == True:
                        continue
                    elif row[key] == False:
                        continue
                    elif row[key] == "#REF!": 
                        continue
                    elif (row["Unit"] == "Time"):
                        continue
                    elif row[key] == "#DIV/0!":
                        continue
                    elif pd.isna(row[key]):
                        continue
                    elif row[key] == '\xa0':
                        continue
                    elif row[key] == int(0) or row[key] == float(0) or row[key] == str(0):
                        continue
                    elif pd.isna(row["Unit"]):
                        continue
                    # if unit is text and value in every column is not string
                    elif (row["Unit"].lower() in ["text"]) and type(row[key]) != str:
                        error_counter += 1
                        outputsheet['C8'].value = f"{outputsheet['C8'].value} Error in Row: {index}, Reference: {df.loc[i, 'Reference']}, Column: {key}, Value: {row[key]}, " \
                            "This is a text field, please check that your input is in text format \n"
                        outputsheet['C8'].fill = red_fill
        if error_counter == 0:
            outputsheet['C8'].value = "Success, No Errors detected!"
            outputsheet['C8'].fill = green_fill 
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {ValueError} \n"
        firstsheet['B6'].fill = red_fill
        pass

In [15]:
def boncode_Consistency(sheet_name,dict_of_sheets,original_dict_of_sheets,excel_error_log_name):

#-----------------------------------------------------------------------------------
#Boncode Consistency*, prevent accidental change/delete of Boncodes
#-----------------------------------------------------------------------------------
    firstsheet = excel_error_log_name['Intro']
    try:
        outputsheet = excel_error_log_name[sheet_name[5:] + ' Output']  # defining which sheet the output needs to go into
        outputsheet['B9'].value = 'Data Validation Rule 7'
        outputsheet['C9'].value = ""
        amended_boncodes=dict_of_sheets['Reference']
        original_boncodes=original_dict_of_sheets['Reference']
        result_string = ''
        original_only_boncodes = list(set(original_boncodes) - set(amended_boncodes))  # The items in the original version, but not in ammended version
        for item in original_only_boncodes:
            if item !=np.nan and item != 'nan':
                result_string += str(item) + '\n'
        if len(original_only_boncodes) > 0:
            outputsheet['C9'].value = f"{outputsheet['C9'].value} Reference (Boncodes) in worksheet in the original version but not in the amended version: {result_string}"
            outputsheet['C9'].fill = red_fill
        else:
            outputsheet['C9'].value = "Success, No Errors detected!"
            outputsheet['C9'].fill = green_fill
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {ValueError} \n"
        firstsheet['B6'].fill = red_fill
        pass

In [16]:
def user_numerical_input(sheet_name, df, excel_error_log_name):
    # ---------------------------------------------------------------------
    ##Whether the values (e.g. anything under the years headers) corresponds to what is in the units columns
    # If the Unit value is not ‘Text’ the years columns values should be number
    #sheet RR 414-419, 489-491 error because unit is weird
    # ---------------------------------------------------------------------
    unit_list = [ '£','000','days','hours','kg','km','kw','l/', 'litres','m2','m3','mg','minutes','ml','months','mtrs','num','tonnes','year','ttds']

    regex_columns = r'^([0-9]{4}-[0-9]{2})$'
    regex_pattern = re.compile(regex_columns)

    firstsheet = excel_error_log_name['Intro']
    try:
        outputsheet = excel_error_log_name[sheet_name[5:] + ' Output']  # defining which sheet the output needs to go into
        outputsheet['B11'].value = 'Data Validation Rule 9'
        outputsheet['C11'].value = ""

        index = 3  # index for showing row position
        error_counter = 0

        for i, row in df.iterrows():
            index += 1
            for key in row.keys():
                if bool(regex_pattern.match(key)) or key.lower() == 'constant':
                    if row[key] == True:
                        continue
                    elif row[key] == False: 
                        continue
                    elif row[key] == "##BLANK":
                        continue
                    elif row[key] == "#DIV/0!":
                        continue
                    elif row[key] == "#REF!": 
                        continue
                    elif pd.isna(row[key]):
                        continue
                    elif row[key] == '\xa0':
                        continue
                    elif row[key] == int(0) or row[key] == float(0) or row[key] == str(0):
                        continue
                    elif pd.isna(row["Unit"]):
                        continue
                    elif (row["Unit"] == "Time"):
                        continue
                    # if unit is numeric, look at the list and value in every column is not int or float
                    else: 
                        for substring in unit_list:
                            if substring in str(row["Unit"]).lower():
                                if isinstance(row[key], str) and row[key].replace('.', '').isdigit():
                                    continue
                                elif type(row[key]) != int and type(row[key]) != float:
                                    #print(f"type: {type(row[key])}, value: {row[key]}, bon: {row['Reference']}, year: {key}  ")
                                    error_counter += 1
                                    outputsheet['C11'].value = f"{outputsheet['C11'].value} Error in Row: {index}, Reference: {df.loc[i, 'Reference']}, Column: {key}, Value: {row[key]}, " \
                                        "This is a numeric field, please check that your input is in numerical format. \n"
                                    outputsheet['C11'].fill = red_fill
        if error_counter == 0:
            outputsheet['C11'].value = "Success, No Errors detected!"
            outputsheet['C11'].fill = green_fill
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {ValueError} \n"
        firstsheet['B6'].fill = red_fill
        pass

In [17]:
def percentage_range(sheet_name, df, excel_error_log_name):
    #checks that entries where the unit is a percentage fall between 0 and 100
    regex_columns = r'^([0-9]{4}-[0-9]{2})$'
    regex_pattern = re.compile(regex_columns)

    firstsheet = excel_error_log_name['Intro']
    try:
        outputsheet = excel_error_log_name[sheet_name[5:] + ' Output']  # defining which sheet the output needs to go into
        outputsheet['B12'].value = 'Data Validation Rule 10 (will not stop data loading to fountain)'
        outputsheet['C12'].value = ""

        index = 3  # index for showing row position
        error_counter = 0

        for i, row in df.iterrows():
            index += 1
            for key in row.keys():
                if bool(regex_pattern.match(key)) or key.lower() == 'constant':
                    if (row["Unit"] == "%"):
                        if type(row[key]) == int or type(row[key]) == float:
                            if row[key]<-2 or row[key]>2:
                                error_counter += 1
                                outputsheet['C12'].value = f"{outputsheet['C12'].value} Error in Row: {index}, Reference: {df.loc[i, 'Reference']}, Column: {key}, Value: {row[key]}, " \
                                    "This is a percentage outside the expected range, with absolute value less than 2, please check that this value is correct. \n"
                                outputsheet['C12'].fill = red_fill        
        if error_counter == 0:
            outputsheet['C12'].value = "Success, No Errors detected!"
            outputsheet['C12'].fill = green_fill
    except ValueError:
        firstsheet['B6'].value = f"{firstsheet['B6'].value} {ValueError} \n"
        firstsheet['B6'].fill = red_fill
        pass

In [22]:
%%time 

original_file_location = v7_bpt

path = input_files

# Change the directory to the O drive
os.chdir(path)

#for file in glob.glob("PR24 Data Tables 240314 variant plan DAM_Amended.xlsx"):  # glob is a way of getting all the files of a certain type
for file in glob.glob(r"*.xlsx"):  # glob is a way of getting all the files of a certain type
    acronym = file[0:3]
    file_location = file
    
    workbook = openpyxl.Workbook()  # creating a new excel sheet to store the data
    sheet = workbook.active  # calling the first sheet
    sheet.title = "Intro"  # defining the first sheet to give the user an introduction
    intro = workbook["Intro"]
    intro['B2'].value = 'This is the introduction sheet, each sheet after this will detail the outputs. This version of Proteus runs against V7 BPT.'
    intro['B8'].value = 'NOTE that we should be expecting to see Boncode errors containing XXX in the OUT and LS_MAN sheets, if you do not see this, make a note in the error log to edit the company specific bons.'
     
    dict_of_sheets={}
    dict_of_sheets=import_data(file_location,workbook)
    original_dict_of_sheets={}
    original_dict_of_sheets=import_original_data(original_file_location,workbook)
    main(dict_of_sheets,original_dict_of_sheets,workbook)
    timestamp_error_log(workbook)
    
    excel_error_log_name = (error_output_path + acronym + " Excel Error log_" + datetime.datetime.now().strftime("%d.%m.%Y %H.%M") + ".xlsx")
    workbook.save(excel_error_log_name)
    print(file)

NotADirectoryError: [WinError 267] The directory name is invalid: 'C:\\Users\\Nahila.Chowdhury\\OneDrive - OFWAT\\Data\\Business Plan Tables\\August 2024 BPT Submission\\3_Files error fixing WORKING\\Additional Tables\\UUWR_91_Additional data tables_DT1.xlsx'

In [19]:
### END OF FUNCTIONS RUN FOR FOUTPUTS 